In [2]:
from rdflib import Graph, URIRef, Namespace
from api.src.schemas.authorities.authority import Authority
from api.src.function.authorities.makeGraph import MakeGraphSubject
from api.src.function.authorities.generateID import GenerateId
from pyfuseki import FusekiUpdate, FusekiQuery
from api.src.routes.translate.makeTranslate import MakeTranslate
from pysolr import Solr
import json
import httpx
from api.src.function.loc.graphExist import GraphExist

In [2]:
upThesaurus = FusekiUpdate('http://localhost:3030', 'authorities')

d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = upThesaurus.run_sparql(d)
response.convert()

solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)
solr.delete(q="*:*",  commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">46</int>\n</lst>\n</response>\n'

In [ ]:
url = 'http://localhost:8000/authorities/loc/subject?uri=http%3A%2F%2Fid.loc.gov%2Fauthorities%2Fsubjects%2Fsh85076502'
r = httpx.get(url)
r.json()

In [5]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}
response = httpx.post(
    'http://localhost:8000/authorities/mads/subject/', headers=headers, json=r.json())
response.status_code

201

In [12]:

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

json_data = {
    'type': 'Topic',
    'adminMetadata': {
        'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
        'descriptionModifier': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
        'creationDate': '2023-05-09',
        'descriptionLanguage': 'http://id.loc.gov/vocabulary/languages/por',
        'generationProcess': 'BiblioKeia v.1',
        'generationDate': '2023-05-09T15:07:54',
        'identifiedBy': [
            {
                'type': 'Lccn',
                'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
                'value': 'sh85076502',
            },
        ],
        'status': {
            'value': 'mstatus:new',
            'label': 'novo',
        },
    },
    'elementList': [
        {
            'type': 'TopicElement',
            'elementValue': {
                'value': 'Bibliotecas',
                'lang': 'pt',
            },
        },
    ],
    'note': None,
    'hasBroaderAuthority': [
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh85038731',
            'label': {
                'value': 'Documentation',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh85108685',
            'label': {
                'value': 'Public institutions',
                'lang': 'en',
            },
        },
    ],
    'hasCloseExternalAuthority': [
        {
            'value': 'http://www.yso.fi/onto/yso/p2787',
            'label': {
                'value': 'kirjastot',
                'lang': 'fi',
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997341',
            'label': {
                'value': 'Libraries',
                'lang': None,
            },
        },
        {
            'value': 'http://d-nb.info/gnd/4006439-6',
            'label': {
                'value': 'Bibliothek',
                'lang': 'de',
            },
        },
        {
            'value': 'http://www.wikidata.org/entity/Q7075',
            'label': {
                'value': 'library',
                'lang': None,
            },
        },
        {
            'value': 'http://data.bnf.fr/ark:/12148/cb13318325d',
            'label': {
                'value': 'Bibliothèques',
                'lang': 'fr',
            },
        },
    ],
    'hasExactExternalAuthority': [
        {
            'value': 'http://lod.nal.usda.gov/nalt/47259',
            'label': {
                'value': 'libraries',
                'lang': 'en',
            },
        },
    ],
    'hasNarrowerAuthority': [
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh85144359',
            'label': {
                'value': 'Volunteer workers in libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh2006003914',
            'label': {
                'value': 'Young adult fiction in libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh85002002',
            'label': {
                'value': 'African Americans and libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh85076716',
            'label': {
                'value': 'Library resources',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh85064278',
            'label': {
                'value': 'Illegal libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh90005545',
            'label': {
                'value': 'Computer specialists in libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh85076583',
            'label': {
                'value': 'Traveling libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh85076560',
            'label': {
                'value': 'International libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh2016001293',
            'label': {
                'value': 'Makerspaces in libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh85076572',
            'label': {
                'value': 'Rental libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh85076567',
            'label': {
                'value': 'Proprietary libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh85123595',
            'label': {
                'value': 'Small libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh2019103075',
            'label': {
                'value': 'Open educational resources in libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh99003406',
            'label': {
                'value': 'Association libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh2003000502',
            'label': {
                'value': 'Joint-use libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh89000080',
            'label': {
                'value': 'Gay libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh85048069',
            'label': {
                'value': 'Fiction in libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh95010034',
            'label': {
                'value': 'Māori (New Zealand people) and libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh2018000796',
            'label': {
                'value': 'Wikis and libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh96009061',
            'label': {
                'value': 'Cambodian Americans and libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh2015000116',
            'label': {
                'value': 'Web applications in libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh85064456',
            'label': {
                'value': 'Imaginary books and libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh85101156',
            'label': {
                'value': 'Photocopying services in libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh86003586',
            'label': {
                'value': 'Hispanic Americans and libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh85076505',
            'label': {
                'value': 'Branch libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh85076573',
            'label': {
                'value': 'Special libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh85113044',
            'label': {
                'value': 'Research libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh85108703',
            'label': {
                'value': 'Public libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh85076584',
            'label': {
                'value': 'Academic libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh2006000644',
            'label': {
                'value': 'Romanies and libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh2002001003',
            'label': {
                'value': 'Bulletin boards in libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh95006771',
            'label': {
                'value': 'Literature in libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh2003004419',
            'label': {
                'value': 'Teenage volunteer workers in libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh95008857',
            'label': {
                'value': 'Digital libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh88004059',
            'label': {
                'value': 'Asian Americans and libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh85076565',
            'label': {
                'value': 'Private libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh87008010',
            'label': {
                'value': 'Telecommunication in libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh85076576',
            'label': {
                'value': 'Subscription libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh99003407',
            'label': {
                'value': 'Volunteer libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh2014002646',
            'label': {
                'value': 'Indian tribal libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh2001006867',
            'label': {
                'value': 'Hungarian Americans and libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh85118345',
            'label': {
                'value': 'School libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh2019000029',
            'label': {
                'value': 'Little free libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh96008065',
            'label': {
                'value': 'German Americans and libraries',
                'lang': 'en',
            },
        },
        {
            'value': 'http://id.loc.gov/authorities/subjects/sh98003888',
            'label': {
                'value': 'Bookbinding libraries',
                'lang': 'en',
            },
        },
    ],
    'hasNarrowerExternalAuthority': [
        {
            'value': 'http://id.worldcat.org/fast/997399',
            'label': {
                'value': 'Libraries--Forecasting',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997379',
            'label': {
                'value': 'Libraries--Cost control',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997481',
            'label': {
                'value': 'Libraries--Technological innovations',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/1353572',
            'label': {
                'value': 'Libraries--Manuscripts',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997395',
            'label': {
                'value': 'Libraries--Endowments',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997435',
            'label': {
                'value': 'Libraries--Safety measures',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997390',
            'label': {
                'value': 'Libraries--Decision making',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997382',
            'label': {
                'value': 'Libraries--Costs',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997487',
            'label': {
                'value': 'Libraries--U.S. states--Technological innovations',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997408',
            'label': {
                'value': 'Libraries--In-service training',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997365',
            'label': {
                'value': 'Libraries--British colonies',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997376',
            'label': {
                'value': 'Libraries--Computer network resources',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997393',
            'label': {
                'value': 'Libraries--Directories',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997484',
            'label': {
                'value': 'Libraries--U.S. states',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997391',
            'label': {
                'value': 'Libraries--Design and construction',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997400',
            'label': {
                'value': 'Libraries--French colonies',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997476',
            'label': {
                'value': 'Libraries--Standards',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997409',
            'label': {
                'value': 'Libraries--Information services',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997342',
            'label': {
                'value': 'Libraries--Accidents',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997485',
            'label': {
                'value': 'Libraries--U.S. states--Automation',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997403',
            'label': {
                'value': 'Libraries--Historiography',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997417',
            'label': {
                'value': 'Libraries--Models',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997380',
            'label': {
                'value': 'Libraries--Cost effectiveness',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997371',
            'label': {
                'value': 'Libraries--Certification',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997421',
            'label': {
                'value': 'Libraries--Psychological aspects',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997374',
            'label': {
                'value': 'Libraries--Collectibles',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997447',
            'label': {
                'value': 'Libraries--Sources',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997378',
            'label': {
                'value': 'Libraries--Congresses',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997375',
            'label': {
                'value': 'Libraries--Communication systems',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997418',
            'label': {
                'value': 'Libraries--Periodicals',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997422',
            'label': {
                'value': 'Libraries--Public opinion',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/1353570',
            'label': {
                'value': 'Libraries--Administration',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997436',
            'label': {
                'value': 'Libraries--Safety measures--Planning',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997412',
            'label': {
                'value': 'Libraries--International cooperation',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997477',
            'label': {
                'value': 'Libraries--Study and teaching',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997394',
            'label': {
                'value': 'Libraries--Economic aspects',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997373',
            'label': {
                'value': 'Libraries--Citizen participation',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997402',
            'label': {
                'value': 'Libraries--Handbooks, manuals, etc.',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997445',
            'label': {
                'value': 'Libraries--Social aspects',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997419',
            'label': {
                'value': 'Libraries--Planning',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997396',
            'label': {
                'value': 'Libraries--Equipment and supplies',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997384',
            'label': {
                'value': 'Libraries--Customer services',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997420',
            'label': {
                'value': 'Libraries--Political aspects',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997483',
            'label': {
                'value': 'Libraries--Transportation',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997381',
            'label': {
                'value': 'Libraries--Cost of operation',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997346',
            'label': {
                'value': 'Libraries--Audio-visual aids',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997410',
            'label': {
                'value': 'Libraries--Information technology',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997478',
            'label': {
                'value': 'Libraries--Study and teaching (Elementary)',
                'lang': None,
            },
        },
        {
            'value': 'http://id.worldcat.org/fast/997434',
            'label': {
                'value': 'Libraries--Research',
                'lang': None,
            },
        },
    ],
    'hasVariant': None,
    'subjectOf': None,
    'isMemberOfMADSCollection': 'http://bibliokeia.com/authorities/Topic/',
}

response = httpx.post('http://localhost:8000/authorities/mads/subject/', headers=headers, json=json_data)
response.status_code

201

In [52]:
with open('exemples/response_1683650145811.json', encoding='utf-8') as file:
    reader = file.read()
    r = json.loads(reader)

In [56]:
r = {"type": "Topic",
 "adminMetadata": {"assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
  "descriptionModifier": "http://id.loc.gov/vocabulary/organizations/brmninpa",
  "creationDate": "2023-05-09",
  "descriptionLanguage": "http://id.loc.gov/vocabulary/languages/por",
  "generationProcess": "BiblioKeia v.1",
  "generationDate": "2023-05-09T15:07:54",
  "identifiedBy": [{"type": "Lccn",
    "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
    "value": "sh85076502"}],
  "status": {"value": "mstatus:new", "label": "novo"}},
 "elementList": [{"type": "TopicElement",
   "elementValue": {"value": "Bibliotecas", "lang": "pt"}}],
 "hasBroaderAuthority": [{"value": "http://id.loc.gov/authorities/subjects/sh85038731",
   "label": {"value": "Documentation", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh85108685",
   "label": {"value": "Public institutions", "lang": "en"}}],
 "hasCloseExternalAuthority": [{"value": "http://www.yso.fi/onto/yso/p2787",
   "label": {"value": "kirjastot", "lang": "fi"}},
  {"value": "http://id.worldcat.org/fast/997341",
   "label": {"value": "Libraries"}},
  {"value": "http://d-nb.info/gnd/4006439-6",
   "label": {"value": "Bibliothek", "lang": "de"}},
  {"value": "http://www.wikidata.org/entity/Q7075",
   "label": {"value": "library"}},
  {"value": "http://data.bnf.fr/ark:/12148/cb13318325d",
   "label": {"value": "Bibliothèques", "lang": "fr"}}],
 "hasExactExternalAuthority": [{"value": "http://lod.nal.usda.gov/nalt/47259",
   "label": {"value": "libraries", "lang": "en"}}],
 "hasNarrowerAuthority": [{"value": "http://id.loc.gov/authorities/subjects/sh85144359",
   "label": {"value": "Volunteer workers in libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh2006003914",
   "label": {"value": "Young adult fiction in libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh85002002",
   "label": {"value": "African Americans and libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh85076716",
   "label": {"value": "Library resources", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh85064278",
   "label": {"value": "Illegal libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh90005545",
   "label": {"value": "Computer specialists in libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh85076583",
   "label": {"value": "Traveling libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh85076560",
   "label": {"value": "International libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh2016001293",
   "label": {"value": "Makerspaces in libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh85076572",
   "label": {"value": "Rental libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh85076567",
   "label": {"value": "Proprietary libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh85123595",
   "label": {"value": "Small libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh2019103075",
   "label": {"value": "Open educational resources in libraries",
    "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh99003406",
   "label": {"value": "Association libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh2003000502",
   "label": {"value": "Joint-use libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh89000080",
   "label": {"value": "Gay libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh85048069",
   "label": {"value": "Fiction in libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh95010034",
   "label": {"value": "Māori (New Zealand people) and libraries",
    "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh2018000796",
   "label": {"value": "Wikis and libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh96009061",
   "label": {"value": "Cambodian Americans and libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh2015000116",
   "label": {"value": "Web applications in libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh85064456",
   "label": {"value": "Imaginary books and libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh85101156",
   "label": {"value": "Photocopying services in libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh86003586",
   "label": {"value": "Hispanic Americans and libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh85076505",
   "label": {"value": "Branch libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh85076573",
   "label": {"value": "Special libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh85113044",
   "label": {"value": "Research libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh85108703",
   "label": {"value": "Public libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh85076584",
   "label": {"value": "Academic libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh2006000644",
   "label": {"value": "Romanies and libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh2002001003",
   "label": {"value": "Bulletin boards in libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh95006771",
   "label": {"value": "Literature in libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh2003004419",
   "label": {"value": "Teenage volunteer workers in libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh95008857",
   "label": {"value": "Digital libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh88004059",
   "label": {"value": "Asian Americans and libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh85076565",
   "label": {"value": "Private libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh87008010",
   "label": {"value": "Telecommunication in libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh85076576",
   "label": {"value": "Subscription libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh99003407",
   "label": {"value": "Volunteer libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh2014002646",
   "label": {"value": "Indian tribal libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh2001006867",
   "label": {"value": "Hungarian Americans and libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh85118345",
   "label": {"value": "School libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh2019000029",
   "label": {"value": "Little free libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh96008065",
   "label": {"value": "German Americans and libraries", "lang": "en"}},
  {"value": "http://id.loc.gov/authorities/subjects/sh98003888",
   "label": {"value": "Bookbinding libraries", "lang": "en"}}],
 "hasNarrowerExternalAuthority": [{"value": "http://id.worldcat.org/fast/997399",
   "label": {"value": "Libraries--Forecasting"}}],
 "isMemberOfMADSCollection": "http://bibliokeia.com/authorities/Topic/"}

In [6]:
from api.src.function.authorities.getUri import GetUri
from api.src.schemas.authorities.authority import Uri, Variant, AdminMetadata, Element
from typing import Optional
from pydantic import BaseModel

class Subject(BaseModel):
    type: str 
    adminMetadata: AdminMetadata 
    elementList: list[Element]
    note: Optional[str]
    hasBroaderAuthority: Optional[list[Uri]]
    hasCloseExternalAuthority: Optional[list[Uri]]
    hasExactExternalAuthority: Optional[list[Uri]]
    hasNarrowerAuthority: Optional[list[Uri]]
    hasNarrowerExternalAuthority: Optional[list[Uri]]
    hasVariant: Optional[list[Variant]]
    subjectOf: Optional[list[Uri]]
    isMemberOfMADSCollection: str

def ParserSubject(graph, authority):
      
  prefix = """PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>"""

  # Type
  qtype = f"""{prefix}
  SELECT ?type WHERE {{ 
      <{authority}> rdf:type ?type .
      FILTER ( ?type != madsrdf:Authority ) 
      FILTER ( ?type != skos:Concept )
       }}"""
  r = graph.query(qtype)
  tipo = r.bindings[0].get('type').split("#")[1]
  

  # adminMetadata
  adminMetadata = {
      "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
      "identifiedBy": [ {
         "type": "Lccn",
          "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
          "value": authority.split('/')[-1]        
      }]      
  }
  # ElementList
  qElementList = f"""{prefix}
  SELECT ?elementValue ?type WHERE  {{
    <{authority}> madsrdf:elementList ?o .
    ?o rdf:rest* ?node .
    ?node rdf:first ?e .
    ?e madsrdf:elementValue ?elementValue .
    ?e rdf:type ?type
    }}"""
  r = graph.query(qElementList)
  elementList = list()
  for i in r.bindings:
      element = {
            "type": i.get('type').split("#")[1],
          "elementValue": {
            "value":  i.get('elementValue').value,
            "lang": i.get('elementValue').language
          }
        }
      elementList.append(element)

  obj = {
     "type": tipo,
      "adminMetadata": adminMetadata,
      "elementList": elementList,
      "isMemberOfMADSCollection": f'http://bibliokeia.com/authorities/{tipo}/'
  }

  # Note 
  qNote = f"""{prefix}
  SELECT ?note WHERE {{ 
      <{authority}> madsrdf:note ?note .
       }}"""
  r = graph.query(qNote)
  if len(r.bindings) > 0:
     obj['note'] = r.bindings[0].get('note').value

# hasVariant
  qVariant = f"""{prefix}
  SELECT ?typeVariant ?typeElement ?elementValue WHERE  {{
	<{authority}> madsrdf:hasVariant ?variant .
  ?variant rdf:type ?typeVariant .
  ?variant madsrdf:elementList ?elementList .
  ?elementList rdf:rest* ?node .
    ?node rdf:first ?e .
    ?e madsrdf:elementValue ?elementValue .
	?e rdf:type ?typeElement .
  FILTER ( ?typeVariant != madsrdf:Variant )
  }}"""
  r = graph.query(qVariant)
  if len(r.bindings) > 0:
    variants = list()
    for i in r.bindings:
      variant = {
          'type': i.get('typeVariant').split("#")[1],
          'elementList': [{
              'type': i.get('typeElement').split("#")[1],
              'elementValue': {
                  'value': i.get('elementValue').value,
                  'lang': i.get('elementValue').language
              }
          }]
      }
      variants.append(variant)
    obj['hasVariant'] = variants

  # URIS
  metadados = ['hasBroaderAuthority', 
               'hasCloseExternalAuthority',
    'hasExactExternalAuthority',
    'hasNarrowerAuthority',
    'hasNarrowerExternalAuthority']
  for metadado in metadados:
     obj = GetUri(obj, metadado, authority, graph)
  
  response = Subject(**obj)
  return response

ModuleNotFoundError: No module named 'src'

In [ ]:
request = Subject(**r.json())
request

In [3]:
graph = Graph()
authority = 'http://id.loc.gov/authorities/subjects/sh85038731'
graph.parse(authority)
# graph.serialize('exemples/documentation.ttl')

<Graph identifier=Ne23b3d7ce6cd4696b6f3fd98cac63395 (<class 'rdflib.graph.Graph'>)>

In [4]:

response = ParserSubject(graph, authority)
response.dict()

NameError: name 'ParserSubject' is not defined

In [48]:
binding['label']['value']

'Bibliotecas'

In [43]:
response.convert()['results']['bindings']

{'head': {'vars': ['uri', 'label']},
 'results': {'bindings': [{'uri': {'type': 'uri',
     'value': 'https://bibliokeia.com/authorities/Topic/bkau-1'},
    'label': {'type': 'literal', 'value': 'Bibliotecas'}}]}}

In [27]:
obj = {
     "type": "Topic",
      "adminMetadata": "adminMetadata",
      "elementList": "elementList",
      "isMemberOfMADSCollection": f'http://bibliokeia.com/authorities/{"tipo"}/'
  }
obj = GetUri(obj, "hasNarrowerAuthority", authority, graph)
obj

{'type': 'Topic',
 'adminMetadata': 'adminMetadata',
 'elementList': 'elementList',
 'isMemberOfMADSCollection': 'http://bibliokeia.com/authorities/tipo/',
 'hasNarrowerAuthority': [{'value': 'http://id.loc.gov/authorities/subjects/sh85102444',
   'label': {'value': 'PIS (Information retrieval system)', 'lang': 'en'}},
  {'value': 'http://id.loc.gov/authorities/subjects/sh85003009',
   'label': {'value': 'AIS (Information retrieval system)', 'lang': 'en'}},
  {'value': 'http://id.loc.gov/authorities/subjects/sh85032443',
   'label': {'value': 'Copying processes', 'lang': 'en'}},
  {'value': 'http://id.loc.gov/authorities/subjects/sh85123620',
   'label': {'value': 'SMART (Information retrieval system)', 'lang': 'en'}},
  {'value': 'http://id.loc.gov/authorities/subjects/sh85066148',
   'label': {'value': 'Information retrieval', 'lang': 'en'}},
  {'value': 'http://id.loc.gov/authorities/subjects/sh2021005166',
   'label': {'value': 'Classification--Books--Documentation', 'lang': 'en'}}

In [25]:
graph

<Graph identifier=N3b3ebc8708944780b54042e56378e903 (<class 'rdflib.graph.Graph'>)>

In [6]:
translator = MakeTranslate(
            source_language='en',
            target_language='pt',
            timeout=10
        )
for k, v in response:
    if k == 'elementList':
        for j in v:
            term = j.elementValue.value
            result = translator.translate(term)
            j.elementValue.value = result.capitalize()
            j.elementValue.lang = 'pt'


In [ ]:
graph = MakeGraphSubject(response, id)
print(graph)

In [17]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'authorities')
response = fuseki_update.run_sparql(graph)
response.convert

<bound method QueryResult.convert of <SPARQLWrapper.Wrapper.QueryResult object at 0x0000025CEBFC4310>>

DOC SOL SUBJECT

In [10]:
from api.src.function.authorities.makeLabel import MakeLabel

In [7]:
request = response

In [19]:
request.hasBroaderAuthority

[Uri(value='http://id.loc.gov/authorities/subjects/sh85038731', label=Label(value='Documentation', lang='en')),
 Uri(value='http://id.loc.gov/authorities/subjects/sh85108685', label=Label(value='Public institutions', lang='en'))]

In [26]:
x = [{'uri': i.value, 'label': i.label.value, 'lang': i.label.lang} for i in request.hasBroaderAuthority]
x

[{'uri': 'http://id.loc.gov/authorities/subjects/sh85038731',
  'label': 'Documentation',
  'lang': 'en'},
 {'uri': 'http://id.loc.gov/authorities/subjects/sh85108685',
  'label': 'Public institutions',
  'lang': 'en'}]

In [40]:
def MakeDocSubject(request, id):

    doc = {
            'id': id,
            'type': request.type,
            "creationDate": request.adminMetadata.creationDate.strftime('%Y-%m-%d'), 
            "label": f'{MakeLabel(request.elementList)}' ,
            "isMemberOfMADSCollection": request.isMemberOfMADSCollection
        }
    for k, v in request:
        if v and k not in ['type', 'adminMetadata', 'elementList', 'isMemberOfMADSCollection']:
                uri = [{'uri': i.value, 'label': i.label.value, 'lang': i.label.lang} for i in v]
                doc[f'{k}'] = uri
    return doc
doc = MakeDocSubject(request, id)

In [43]:
solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)
responseSolr = solr.add([doc], commit=True)
responseSolr

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":821}}\n'

SIMPLE TYPE

In [3]:
uri = URIRef('http://id.loc.gov/authorities/subjects/sh85076502')

In [6]:
uri = 'http://id.loc.gov/authorities/subjects/sh85076502'
prefix = """PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>"""
qlccn = f"""{prefix}
SELECT ?o WHERE {{ 
    <{uri}> identifiers:lccn ?o }}"""

qType = f"""{prefix}
SELECT ?o WHERE {{ 
    <{uri}> rdf:type ?o }}"""

qElementList = f"""{prefix}
SELECT ?elementValue ?type WHERE  {{
	<{uri}> madsrdf:elementList ?o .
  ?o rdf:rest* ?node .
  ?node rdf:first ?e .
  ?e madsrdf:elementValue ?elementValue .
  ?e rdf:type ?type
  }}"""

qComponentList = f"""{prefix}
SELECT * WHERE  {{
	<{uri}> madsrdf:componentList ?componentList .
  }}"""

qFullerName = f"""{prefix}
SELECT * WHERE  {{
	<{uri}> madsrdf:fullerName ?fullerName .
  }}"""

qBroader = f"""{prefix}
SELECT * WHERE  {{
	<{uri}> madsrdf:hasBroaderAuthority ?value .
  ?value madsrdf:authoritativeLabel ?label
  }}"""

qCloseExternal = f"""{prefix}
SELECT * WHERE  {{
	<{uri}> madsrdf:hasCloseExternalAuthority ?value .
  ?value madsrdf:authoritativeLabel ?label
  }}"""

qExactExternal = f"""{prefix}
SELECT * WHERE  {{
	<{uri}> madsrdf:hasExactExternalAuthority ?value .
  ?value madsrdf:authoritativeLabel ?label
  }}"""

qNarrower = f"""{prefix}
SELECT * WHERE  {{
	<{uri}> madsrdf:hasNarrowerAuthority ?value .
  ?value madsrdf:authoritativeLabel ?label
  }}"""





In [4]:
def getUri(obj, metadado, authority, graph):
    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:{metadado} ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        uris = list()
        for i in response.bindings:
          uri = {
                  "value": i.get('value').n3().replace('<','').replace('>',''),
              "label": {
                "value": i.get('label').value,
                "lang": i.get('label').language
              } }
          uris.append(uri)
        obj[metadado] = uris
    return obj

obj = {}
uri = 'http://id.loc.gov/authorities/subjects/sh85076502'
obj = getUri(obj, "hasBroaderAuthority", uri, g)
obj


{'hasBroaderAuthority': [{'value': 'http://id.loc.gov/authorities/subjects/sh85038731',
   'label': {'value': 'Documentation', 'lang': 'en'}},
  {'value': 'http://id.loc.gov/authorities/subjects/sh85108685',
   'label': {'value': 'Public institutions', 'lang': 'en'}}]}

In [15]:
obj = {
    'type': [i.split("#")[1] for [i] in g.query(qType) ][0],
    "adminMetadata": {
    "generationProcess": {
      "label": "BiblioKeia Beta"
    },
    "identifiedBy": {
      "assigner": "http://id.loc.gov/vocabulary/organizations/brmninpa",
      "value": id
    }
  }, 
  "isMemberOfMADSCollection": "http://id.loc.gov/authorities/subjects/"
    }

[obj['lccn']] = [i[0].value for i in g.query(qlccn) ]

# elementList
response = g.query(qElementList)
if len(response.bindings) > 0:
    elementList = list()
    for i in response.bindings:
      element = {
          "type": i.get('type').split("#")[1],
        "elementValue": {
          "value":  i.get('elementValue').value,
          "lang": i.get('elementValue').language
        }
      }
      elementList.append(element)
    obj['elementList'] = elementList
  
# ComponentList
response = g.query(qComponentList)
if len(response.bindings) > 0:
    print("qComponentList")

# fullerName
response = g.query(qComponentList)
if len(response.bindings) > 0:
    print("fullerName")

# birthDate: Optional[str]
#     birthPlace: Optional[Uri]
#     deathDate: Optional[str]
#     occupation: Optional[list[Uri]]
#     hasAffiliation: Optional[list[Affiliation]]

# hasBroaderAuthority
obj = getUri(obj, "hasBroaderAuthority", uri, g)

# hasCloseExternalAuthority
obj = getUri(obj, "hasCloseExternalAuthority", uri, g)

# hasExactExternalAuthority
obj = getUri(obj, "hasExactExternalAuthority", uri, g)

# hasNarrowerAuthority
obj = getUri(obj, "hasExactExternalAuthority", uri, g)

# hasVariant: 




request = Authority(**obj)
request
# obj

Authority(type='Topic', identifiers=None, adminMetadata=AdminMetadata(assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate=datetime.date(2023, 5, 2), descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess=GenerationProcess(label='BiblioKeia Beta', generationDate='2023-05-02T19:56:44'), identifiedBy=IdentifiedBy(assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', value='bkau-1'), status=Status(value='mstatus:new', label='novo')), elementList=[Element(type='TopicElement', elementValue=Label(value='Libraries', lang='en'))], componentList=None, fullerName=None, birthDate=None, birthPlace=None, deathDate=None, occupation=None, hasAffiliation=None, hasBroaderAuthority=[Uri(value='http://id.loc.gov/authorities/subjects/sh85038731', label=Label(value='Documentation', lang='en')), Uri(value='http://id.loc.gov/authorities/subjects/sh85108685', label=Label(va

In [16]:
graph = MakeGraph(request, id)
print(graph)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#> 
    PREFIX owl: <http://www.w3.org/2002/07/owl#> 
    PREFIX ri: <http://id.loc.gov/ontologies/RecordInfo#> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/> 
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/> 
    PREFIX bkw: <https://bibliokeia.com/resources/work/> 
    PREFIX menclvl: <http://id.loc.gov/vocabulary/menclvl/>
    PREFIX mstatus: <http://id.loc.gov/vocabulary/mstatus/>
    PREFIX contentTypes: <http://id.loc.gov/vocabulary/contentTypes/>
    PREFIX relators: <http://id.loc.gov/vocabulary/relators/>
    PREFIX genreForms: <http://id.loc.gov/authorities/genreForms/>
    PREFIX msupplcont: <http://id.loc.gov/vocabulary/msupplcont/>
    PREFIX millus: <http://id.loc.gov/vocabul

In [20]:
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [126]:
response = g.query(qElementList)
for i in response.bindings:
    element = {
         "type": i.get('type').split("#")[1],
      "elementValue": {
        "value":  i.get('elementValue').value,
        "lang": i.get('elementValue').language
      }
    }
    print(element)

{'type': 'TopicElement', 'elementValue': {'value': 'Libraries', 'lang': 'en'}}


In [125]:
x = i.get('type').split("#")[1]
x

'TopicElement'

In [84]:
graph = MakeGraph(request, 'bk-TESTE')
print(graph)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#> 
    PREFIX owl: <http://www.w3.org/2002/07/owl#> 
    PREFIX ri: <http://id.loc.gov/ontologies/RecordInfo#> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/> 
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/> 
    PREFIX bkw: <https://bibliokeia.com/resources/work/> 
    PREFIX menclvl: <http://id.loc.gov/vocabulary/menclvl/>
    PREFIX mstatus: <http://id.loc.gov/vocabulary/mstatus/>
    PREFIX contentTypes: <http://id.loc.gov/vocabulary/contentTypes/>
    PREFIX relators: <http://id.loc.gov/vocabulary/relators/>
    PREFIX genreForms: <http://id.loc.gov/authorities/genreForms/>
    PREFIX msupplcont: <http://id.loc.gov/vocabulary/msupplcont/>
    PREFIX millus: <http://id.loc.gov/vocabul

In [5]:
g.serialize('libraries.ttl', format='ttl')

<Graph identifier=N8c2571aa3f5546fdbfcdb568212cd7fa (<class 'rdflib.graph.Graph'>)>